In [1]:
# !pip install pyshp
import pandas as pd
import pickle
from scipy.spatial import cKDTree
import numpy as np

def read_shapefile(shp_path):
    """
    Read a shapefile into a Pandas dataframe with a 'coords' column holding
    the geometry information. This uses the pyshp package
    """
    import shapefile

    #read file, parse out the records and shapes
    sf = shapefile.Reader(shp_path)
    fields = [x[0] for x in sf.fields][1:]
    records = sf.records()
    shps = [s.points for s in sf.shapes()]

    #write into a dataframe
    df = pd.DataFrame(columns=fields, data=records)
    df = df.assign(coords=shps)

    return df

In [2]:
df = read_shapefile('data/outdoor_inventory/Outdoor_Inventory_AV.shp')

In [3]:
reverse_coords_list = pd.read_pickle('data/reverse_loc.pickle')

In [4]:
df['address_road'] = [x.get('address').get('road') for x in reverse_coords_list]
df['concelho'] = [x.get('address').get('town') for x in reverse_coords_list]
df['freguesia'] = [x.get('address').get('village') if 'village' in x.get('address') else x.get('address').get('neighbourhood') for x in reverse_coords_list]
df['distrito'] = [x.get('address').get('county') for x in reverse_coords_list]

In [5]:
df['address_road'] = df['address_road'].fillna('')
df['is_national_road'] = df['address_road'].str.contains('EN')
df['is_highway'] = df['address_road'].str.contains('Auto')
df['is_city_center'] = (~df['is_national_road']) & (~df['is_highway']) & (df['address_road'] != '')

In [6]:
df.head(3)

,PanelID,X,Y,Max_Visibi,Average_Da,coords,address_road,concelho,freguesia,distrito,is_national_road,is_highway,is_city_center
0,26144,-8.473975,40.900768,69,31.0,"[[-8.473975, 40.900768]]",Rua Alto das Casas,São João da Madeira,Macieira de Sarnes,Aveiro,False,False,True
1,11714,-9.315949,38.958125,69,31.0,"[[-9.315949, 38.958125]]",EN 9,Mafra,Barreiralva,Lisboa,True,False,False
2,26109,-8.510079,40.871821,69,32.0,"[[-8.510079, 40.871821]]",Rua Professor Doutor António Joaquim Ferreira ...,Oliveira de Azeméis,Vila de Cucujães,Aveiro,False,False,True


In [7]:
# https://geoffboeing.com/2014/08/clustering-to-reduce-spatial-data-set-size/
# https://stackoverflow.com/questions/43592094/efficient-way-to-calculate-geographic-density-in-pandas 
# https://stackoverflow.com/questions/34579213/dbscan-for-clustering-of-geographic-location-data 

def find_neighbours_within_radius(xy, radius):
    tree = cKDTree(xy)
    within_radius = tree.query_ball_tree(tree, r=radius)
    return within_radius

In [8]:
def get_density_billboards(df):
    """
    Calculate number of billboards in a X km radius for each point
    """
    
    df_copy = df.copy()

    import numpy as np

    kms_per_radian = 6371.0088
    radius = 1.5
    neighbours_within_radius = find_neighbours_within_radius(np.radians(df_copy[['Y', 'X']].values), radius/kms_per_radian)

    df_copy['nbr_points_around_billboard'] = [len(x) for x in neighbours_within_radius]

    return df_copy, neighbours_within_radius

In [9]:
df, neighbours_within_radius = get_density_billboards(df)

In [10]:
neighbours_within_radius[0]

[0,
 792,
 1060,
 1686,
 4455,
 5095,
 5225,
 5485,
 7093,
 7362,
 8007,
 8703,
 8933,
 12413,
 12768,
 13477,
 15485,
 16590,
 18603,
 19241,
 21358,
 21619,
 24057,
 24927,
 28937]

In [11]:
from geopy import distance
df.iloc[0][['X', 'Y', 'freguesia']], df.iloc[24][['X', 'Y', 'freguesia']], distance.distance((38.9581, -9.31595), (38.9639, -9.31799))

(X                     -8.473975
 Y                     40.900768
 freguesia    Macieira de Sarnes
 Name: 0, dtype: object,
 X            -8.432738
 Y            40.214686
 freguesia         None
 Name: 24, dtype: object,
 Distance(0.6677214916495565))

In [12]:
df['average_people_around_billboard'] = [np.mean(df.iloc[x].Average_Da) for x in neighbours_within_radius]
df['std_people_around_billboard'] = [np.std(df.iloc[x].Average_Da) for x in neighbours_within_radius]

# points where this value is very negative are just visual noise, they make no impact
# retirar N billboards por concelho
df['diff_to_neighbourhood'] = df['Average_Da'] - df['average_people_around_billboard']

# sitios maus: sitios com pouca gente, e que tens muita gente a volta. As pessoas estão a passar pelos outros, e não pelo teu. 
# zonas com muita densidade: estamos a dividir a eficiencia de marketing com outras empresas

# Locais para novos billboards?

In [13]:
df[df.diff_to_neighbourhood < 0].head(3)

,PanelID,X,Y,Max_Visibi,Average_Da,coords,address_road,concelho,freguesia,distrito,is_national_road,is_highway,is_city_center,nbr_points_around_billboard,average_people_around_billboard,std_people_around_billboard,diff_to_neighbourhood
0,26144,-8.473975,40.900768,69,31.0,"[[-8.473975, 40.900768]]",Rua Alto das Casas,São João da Madeira,Macieira de Sarnes,Aveiro,False,False,True,25,5148.240000,4158.391442,-5117.240000
1,11714,-9.315949,38.958125,69,31.0,"[[-9.315949, 38.958125]]",EN 9,Mafra,Barreiralva,Lisboa,True,False,False,9,998.333333,1228.854028,-967.333333
2,26109,-8.510079,40.871821,69,32.0,"[[-8.510079, 40.871821]]",Rua Professor Doutor António Joaquim Ferreira ...,Oliveira de Azeméis,Vila de Cucujães,Aveiro,False,False,True,5,92.000000,44.181444,-60.000000


In [14]:
df[df.nbr_points_around_billboard > 200].head(3)

,PanelID,X,Y,Max_Visibi,Average_Da,coords,address_road,concelho,freguesia,distrito,is_national_road,is_highway,is_city_center,nbr_points_around_billboard,average_people_around_billboard,std_people_around_billboard,diff_to_neighbourhood
9,34634,-9.143221,38.710728,63,34.0,"[[-9.143221, 38.710728]]",Praça Luís de Camões,None,Encarnação,Lisboa,False,False,True,597,10221.666108,8226.519490,-10187.666108
17,31707,-8.673731,41.179162,69,36.0,"[[-8.673731, 41.179162]]",Avenida Vilagarcía de Arousa,Matosinhos,Real,Porto,False,False,True,303,6715.917492,5454.125083,-6679.917492
36,49291,-9.204441,38.710662,71,43.0,"[[-9.204441, 38.710662]]",Avenida Helen Keller,None,None,Lisboa,False,False,True,303,8898.735974,6511.957892,-8855.735974


# Metaheuristics: Billboard optimization

In [15]:
# Focus on lisbon

df = df[df['distrito'] == 'Lisboa'].reset_index()

In [16]:
def fitness(df):
    """
    Fitness function, which we want to minimize
    """
    
    # Get billboard density
    df, _ = get_density_billboards(df)

    max_density = df['nbr_points_around_billboard'].mean()
    total_number_of_views = df['Average_Da'].sum()
    return max_density # + total_number_of_views

In [17]:
fitness(df)

382.6084085051546

In [18]:
# We might have different impressions for the same coordinate (maybe diff directions of traffic?) so let's average it out

MEAN_IMPRESSIONS_PER_COORD = df.groupby(['X', 'Y']).Average_Da.mean().reset_index()

In [21]:
df.shape, len(df)

((12416, 18), 12416)

In [20]:
def get_neighbours(df, nbr_neighbours):
    """
    Creates a list of size nbr_neighbours with potential swaps of coordinates.
    
    It first picks random samples from MEAN_IMPRESSIONS_PER_COORD, random positions in df and allocates those values.
    """
    
    neighbors = []
    
    # Pick new positions for swapping random elements of df
    sampled_coords = MEAN_IMPRESSIONS_PER_COORD.sample(nbr_neighbours).reset_index()
    
    for i in range(nbr_neighbours):
        
        df_copy = df.copy()

        # Pick a random number between 0 and len(df)
        random_number = np.random.randint(0, len(df_copy))
                    
        # Replace by the Average_Da at that position
        df_copy.loc[random_number, ['X', 'Y', 'Average_Da']] = sampled_coords.loc[i, ['X', 'Y', 'Average_Da']]
                
        neighbors.append(df_copy)
    
    return neighbors

In [30]:
def shuffle_perturbation(neighbor, prob=0.1):
    """
    Randomly shuffles a subset of positions in the generated neighbors.
    
    It picks a neighbor and generates a new neighbour with some of its positions shuffled according to a given probability.
    """
    
    # Create a copy of the neighbor
    neighbor_copy = neighbor.copy()
    
    
    # Get the length of this neighbor
    neighbor_len = len(neighbor_copy)
    
    
    # Create a mask of the positions that will shuffle
    mask = np.random.random_sample((neighbor_len,)) <= prob
    mask = np.arange(neighbor_len)[mask]
    
    
    # Create a shuffled mask to change within these values
    shuffled_mask = mask.copy()
    
    # Shuffle this mask
    np.random.shuffle(shuffled_mask)
    
    # TODO: Create a list with assessed values to avoid "overshuffling"
    
    # TODO: This is only for debugging purposes for now
    neighbor_copy.loc[mask, ['X', 'Y', 'Average_Da']] = neighbor_copy.loc[shuffled_mask, ['X', 'Y', 'Average_Da']]
    
    return neighbor_copy

"""
To implement:
    Neighborhood that 
    floor assignment
    
class ShufflePerturbation(Neighborhood):
    def __init__(self, prob=0.1):
        self.prob = prob

    def compute(self, solution):
        ret = solution.clone()

        maxlen = len(ret.assignment)

        mask = np.random.random(maxlen) <= self.prob
        mask = np.arange(maxlen)[mask]

        values = ret.assignment[mask]
        np.random.shuffle(values)

        ret.assignment[mask] = values

        return [ret]
        
"""

'\nTo implement:\n    Neighborhood that \n    floor assignment\n    \nclass ShufflePerturbation(Neighborhood):\n    def __init__(self, prob=0.1):\n        self.prob = prob\n\n    def compute(self, solution):\n        ret = solution.clone()\n\n        maxlen = len(ret.assignment)\n\n        mask = np.random.random(maxlen) <= self.prob\n        mask = np.arange(maxlen)[mask]\n\n        values = ret.assignment[mask]\n        np.random.shuffle(values)\n\n        ret.assignment[mask] = values\n\n        return [ret]\n        \n'

In [31]:
# TODO: Debugging stuff, erase uppon review
# mask = np.random.random_sample((100,)) <= 0.1
# print(mask)
# mask = np.arange(100)[mask]
# shuffled_mask = mask.copy() 
# np.random.shuffle(shuffled_mask)
# mask, shuffled_mask

In [32]:
neighbors_it_1 = get_neighbours(df, 5)

In [33]:
fitness(neighbors_it_1[0]), fitness(neighbors_it_1[1]), fitness(neighbors_it_1[3])

(382.62709407216494, 382.6314432989691, 382.6578608247423)

Optimization

Gets neighbours in the best solution

In [ ]:
best_fitness = np.inf
best_solution = df

history_of_best_solutions = []

for iteration in range(10):
    
    # Get 5 random neighbours
    neighbors = get_neighbours(best_solution, 5)
    
    # Get 5 more shuffled neighbours and append to these neighbors list
    for n in neighbors:
        neighbors.append(shuffle_perturbation(n))
    
    
    # Sanity-check print
    print(f"Nr of possible solutions: {len(neighbors)}.")
    
    
    # Get fitnesses of possible solutions
    fitness_neighbours = [fitness(x) for x in neighbors]
    
    neighbour_lower_fitness_idx = np.argmin(fitness_neighbours)
    fitness_neighbour_lower_fitness = fitness_neighbours[neighbour_lower_fitness_idx]
    neighbour_lower_fitness = neighbors[neighbour_lower_fitness_idx]
    
    if fitness_neighbour_lower_fitness < best_fitness:
        best_fitness = fitness_neighbour_lower_fitness
        best_solution = neighbour_lower_fitness
        
        history_of_best_solutions.append(best_solution)

        print("Found a better solution!")
        
    print("Epoch %d | Fitness %f" % (iteration, best_fitness))

Check changes

In [36]:
COLS = ['PanelID', 'X', 'Y', 'Max_Visibi']

In [37]:
diff = pd.concat([df[COLS], best_solution[COLS]]).drop_duplicates(keep=False)
diff.head(3)

,PanelID,X,Y,Max_Visibi
5470,29196,-9.148472,38.725948,69
5750,23518,-9.129396,38.741454,69
7199,24126,-9.151310,38.708844,69


In [42]:
df[df.PanelID == 24126]

,PanelID,X,Y,Max_Visibi,Average_Da,coords,address_road,concelho,freguesia,distrito,is_national_road,is_highway,is_city_center,nbr_points_around_billboard,average_people_around_billboard,std_people_around_billboard,diff_to_neighbourhood
7199,24126,-9.15131,38.708844,69,3079.0,"[[-9.15131, 38.708844]]",Largo do Conde Barão,None,Santa Catarina,Lisboa,False,False,True,582,9944.965206,8324.413258,-6865.965206


In [43]:
best_solution[best_solution.PanelID == 24126]

,PanelID,X,Y,Max_Visibi,Average_Da,coords,address_road,concelho,freguesia,distrito,is_national_road,is_highway,is_city_center,nbr_points_around_billboard,average_people_around_billboard,std_people_around_billboard,diff_to_neighbourhood
7199,24126,-9.358457,38.698971,69,5267.0,"[[-9.15131, 38.708844]]",Largo do Conde Barão,None,Santa Catarina,Lisboa,False,False,True,582,9944.965206,8324.413258,-6865.965206


In [44]:
fitness(best_solution)

231.43061728395062

In [45]:
fitness(df)

231.69563786008231

In [ ]:
# Ao trocar da posição (x1, y1) para (x2, y2) estou a ganhar X de audiencia
# O custo médio de aluguer é X€ , mantém se fixo
# assumindo y% das pessoas que vêm têm interesse na marca, estamos a ter um impacto K vezes ... 